# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
current_directory = os.getcwd()
csvpath = os.path.join(current_directory,'..', 'WeatherPy', 'output_data', 'weather_data.csv')

weather_data = pd.read_csv(csvpath)
weather_data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,upernavik,72.7868,-56.1549,-2.22,88,100,2.03,GL,1619233749
1,1,bluff,-46.6000,168.3333,11.11,90,64,0.45,NZ,1619233738
2,2,klaksvik,62.2266,-6.5890,6.00,100,90,2.57,FO,1619233642
3,3,calabozo,8.9242,-67.4293,31.93,37,87,3.69,VE,1619233874
4,4,bambous virieux,-20.3428,57.7575,24.44,94,75,5.14,MU,1619233840


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [37]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations & convert to float
locations = weather_data[["Lat", "Lng"]].astype(float)

#Convert Humidity into float value
humidity = weather_data["Humidity"].astype(float)

# Customize the size of the figure
figure_layout = {
    'width': '750px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    }

# Create a poverty Heatmap layer
fig = gmaps.figure(layout=figure_layout, center=(20.0, -4.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', padding='1px', width='750px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
best_weather = weather_data.loc[weather_data["Humidity"]>=90]
best_weather = best_weather.loc[best_weather["Max Temp"]>=25]

best_weather.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,georgetown,5.4112,100.3354,28.00,94,40,2.06,MY,1619233875
119,119,cayenne,4.9333,-52.3333,25.56,94,75,1.54,GF,1619233829
124,124,puerto ayora,-0.7393,-90.3518,25.00,95,13,1.79,EC,1619233847
192,192,touros,-5.1989,-35.4608,25.00,94,40,2.06,BR,1619233886


In [5]:
# Store into variable named `hotel_df`.
hotel_df = best_weather

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# geocoordinates
target_search = "hotel"
target_radius = 5000
target_type = "lodging"

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "key": g_key,
    "keyword": target_search,
    "type": target_type,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get city coordinates
    hotel_coords = str(row["Lat"]) + "," + str(row["Lng"])
 
    # add keyword to params dic
    params['location'] = hotel_coords

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
#     print(response)
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

hotel_df.head()

Closest hotel is Eastern & Oriental Hotel, Penang.
------------
Closest hotel is Mercure Cayenne Royal Amazonia.
------------
Closest hotel is Hotel Solymar Galapagos.
------------
Closest hotel is Hotel Vila Gale Touros.
------------


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
8,8,georgetown,5.4112,100.3354,28.00,94,40,2.06,MY,1619233875,"Eastern & Oriental Hotel, Penang"
119,119,cayenne,4.9333,-52.3333,25.56,94,75,1.54,GF,1619233829,Mercure Cayenne Royal Amazonia
124,124,puerto ayora,-0.7393,-90.3518,25.00,95,13,1.79,EC,1619233847,Hotel Solymar Galapagos
192,192,touros,-5.1989,-35.4608,25.00,94,40,2.06,BR,1619233886,Hotel Vila Gale Touros


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [31]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(hotel_df[["Lat", "Lng"]],
                            info_box_content=hotel_info)
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(markers)

fig

# Display figure


Figure(layout=FigureLayout(height='420px'))